In [1]:
import sys
import os
import tensorflow as tf
import numpy as np
import librosa
import math
from time import time
sys.path.append('..')
from wavenet.model import WaveNetModel
from wavenet.ops import mu_law_encode, mu_law_decode
from IPython.display import Audio

%matplotlib inline

In [2]:
tf.reset_default_graph()
batch_size = 1
filter_width = 3
n_stack = 2
max_dilation = 10
dilations = [2 ** i for j in range(n_stack) for i in range(max_dilation)]

residual_channels, dilation_channels, skip_channels = 128, 128, 256
use_biases = True
quantization_channels = 256
gc_cardinality = None
gc_channels = None
scalar_input = False
initial_filter_width = filter_width

net = WaveNetModel(batch_size=batch_size,
                        dilations=dilations,
                        filter_width=filter_width,
                        scalar_input=scalar_input,
                        initial_filter_width=initial_filter_width,
                        residual_channels=residual_channels,
                        dilation_channels=dilation_channels,
                        quantization_channels=quantization_channels,
                        skip_channels=skip_channels,
                        global_condition_channels=gc_channels,
                        global_condition_cardinality=gc_cardinality,
                        use_biases=use_biases,
                        local_condition_channels=1)

gen_num = tf.placeholder(tf.int32)
input_batch = tf.placeholder(tf.float32)
lc_batch = tf.placeholder(tf.float32)
ml_encoded = mu_law_encode(input_batch, quantization_channels)
encoded = net._one_hot(ml_encoded)

raw_output = net.create_network(encoded, lc_batch, None)
out = tf.reshape(raw_output, [-1, quantization_channels])
proba = tf.cast(tf.nn.softmax(tf.cast(out, tf.float64)), tf.float32)
# loss = net.loss(input_placeholder, None, None)
# optimizer = tf.train.AdamOptimizer(0.001)
# optim = optimizer.minimize(loss, var_list=tf.trainable_variables())

# For generation
generation_batch_size = 1
sample_placeholder = tf.placeholder(tf.int32)
lc_placeholder = tf.placeholder(tf.float32)
gen_num = tf.placeholder(tf.int32)

next_sample_prob, layers_out, qs = \
    net.predict_proba_incremental(sample_placeholder, gen_num, batch_size=generation_batch_size,
                                 local_condition=lc_placeholder)

initial = tf.placeholder(tf.float32)
others = tf.placeholder(tf.float32)
update_q_ops = net.create_update_q_ops(qs, initial, others, gen_num, batch_size=generation_batch_size)

var_q = net.get_vars_q()

print("created.")

created.


In [3]:
src, _ = librosa.load("voice.wav", sr=16000)
src = src[:len(src)//4]
n_samples = len(src)
src = src.reshape(-1, 1)
src = np.pad(src, [[net.receptive_field, 0], [0, 0]],'constant')

In [4]:
sess_config = tf.ConfigProto(
    device_count = {'GPU': 0}
)
with tf.Session(config=sess_config) as sess:
    sess.run(tf.global_variables_initializer())
    _lc = src.reshape(1, -1, 1)
    result, _encoded = sess.run([proba, ml_encoded], 
                                   feed_dict={input_batch:src, lc_batch:_lc})
    _encoded = _encoded.reshape(batch_size, -1)
    result = np.argmax(result, axis=-1)

    sess.run(tf.variables_initializer(var_q))
    
    t = time()
    samples= []
    for j in range(net.receptive_field-1):
        feed_dict = {sample_placeholder:_encoded[:,j], lc_placeholder:[[0]], gen_num:j}
        prob, _layers = sess.run([next_sample_prob, layers_out], feed_dict=feed_dict)
        sess.run(update_q_ops, feed_dict={initial:_layers[0], others:np.array(_layers[1:]), gen_num:j})

    for j in range(net.receptive_field-1, _encoded.shape[-1]):
        feed_dict = {sample_placeholder:_encoded[:,j], 
                     lc_placeholder:_lc[:,j],
                     gen_num:j}
        prob, _layers = sess.run([next_sample_prob, layers_out], feed_dict=feed_dict)
        sess.run(update_q_ops, feed_dict={initial:_layers[0], others:np.array(_layers[1:]), gen_num:j})
        sample = np.argmax(prob, axis=-1)
        samples.append(sample)
        
    samples = np.array(samples).T
    print("elapsed:", time()-t)

print("result:", result)
print("generated samples:", samples)
print("difference between result and samples:", np.abs(result-samples).sum())

elapsed: 32.192336082458496
result: [255 255 255 ...  19   1  57]
generated samples: [[255 255 255 ...  19   1  57]]
difference between result and samples: 0
